In [13]:
import os
import gc
import time
import math
import datetime
from math import log, floor
from sklearn.neighbors import KDTree

import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle
from tqdm.notebook import tqdm as tqdm

import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad

import scipy
import statsmodels
from scipy import signal
import statsmodels.api as sm
from prophet import Prophet
from scipy.signal import butter, deconvolve
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

import warnings
warnings.filterwarnings("ignore")

In [14]:
SALES_PATH = '/ssd003/projects/forecasting_bootcamp/bootcamp_datasets/m5-forecasting-accuracy/sales_train_evaluation.csv'
CALENDAR_PATH = '/ssd003/projects/forecasting_bootcamp/bootcamp_datasets/m5-forecasting-accuracy/calendar.csv'
PRICES_PATH = '/ssd003/projects/forecasting_bootcamp/bootcamp_datasets/m5-forecasting-accuracy/sell_prices.csv'

sales_train_val = pd.read_csv('/ssd003/projects/forecasting_bootcamp/bootcamp_datasets/m5-forecasting-accuracy/sales_train_evaluation.csv')
sales_train_val.name = 'sales'
calendar = pd.read_csv('/ssd003/projects/forecasting_bootcamp/bootcamp_datasets/m5-forecasting-accuracy/calendar.csv')
calendar.name = 'calendar'
selling_prices = pd.read_csv('/ssd003/projects/forecasting_bootcamp/bootcamp_datasets/m5-forecasting-accuracy/sell_prices.csv')
selling_prices.name = 'prices'

In [15]:
#Train Val Split

d_cols = [c for c in sales_train_val.columns if 'd_' in c]
train_dataset = sales_train_val[d_cols[-100:-30]]
val_dataset = sales_train_val[d_cols[-30:]]
n_rows=25

In [16]:
#Naive Approach

predictions = []
for i in range(len(val_dataset.columns)):
    if i == 0:
        predictions.append(train_dataset[train_dataset.columns[-1]].values)
    else:
        predictions.append(val_dataset[val_dataset.columns[i-1]].values)
    
predictions = np.transpose(np.array([row.tolist() for row in predictions]))
error_naive = np.linalg.norm(predictions[:n_rows] - val_dataset.values[:n_rows])/len(predictions[0])


In [17]:
#Moving Average 

predictions = []
for i in range(len(val_dataset.columns)):
    if i == 0:
        predictions.append(np.mean(train_dataset[train_dataset.columns[-30:]].values, axis=1))
    if i < 31 and i > 0:
        predictions.append(0.5 * (np.mean(train_dataset[train_dataset.columns[-30+i:]].values, axis=1) + \
                                  np.mean(predictions[:i], axis=0)))
    if i > 31:
        predictions.append(np.mean([predictions[:i]], axis=1))
    
predictions = np.transpose(np.array([row.tolist() for row in predictions]))
error_avg = np.linalg.norm(predictions[:n_rows] - val_dataset.values[:n_rows])/len(predictions[0])


In [18]:
#Holt Linear

predictions = []
for row in tqdm(train_dataset[train_dataset.columns[-30:]].values[:n_rows]):
    fit = Holt(row).fit(smoothing_level = 0.3, smoothing_slope = 0.01)
    predictions.append(fit.forecast(30))
predictions = np.array(predictions).reshape((-1, 30))
error_holt = np.linalg.norm(predictions - val_dataset.values[:len(predictions)])/len(predictions[0])


  0%|          | 0/25 [00:00<?, ?it/s]

In [19]:
#Exponential Smoothing

predictions = []
for row in tqdm(train_dataset[train_dataset.columns[-30:]].values[:n_rows]):
    fit = ExponentialSmoothing(row, seasonal_periods=3).fit()
    predictions.append(fit.forecast(30))
predictions = np.array(predictions).reshape((-1, 30))
error_exponential = np.linalg.norm(predictions[:n_rows] - val_dataset.values[:n_rows])/len(predictions[0])


  0%|          | 0/25 [00:00<?, ?it/s]

In [20]:
#ARIMA

predictions = []
for row in tqdm(train_dataset[train_dataset.columns[-30:]].values[:n_rows]):
    fit = sm.tsa.statespace.SARIMAX(row, seasonal_order=(0, 1, 1, 7)).fit()
    predictions.append(fit.forecast(30))
predictions = np.array(predictions).reshape((-1, 30))
error_arima = np.linalg.norm(predictions[:n_rows] - val_dataset.values[:n_rows])/len(predictions[0])


  0%|          | 0/25 [00:00<?, ?it/s]

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.51391D+00    |proj g|=  3.29224D-01


 This problem is unconstrained.



At iterate    5    f=  1.37351D+00    |proj g|=  1.59405D-02

At iterate   10    f=  1.37241D+00    |proj g|=  2.36062D-03

At iterate   15    f=  1.37232D+00    |proj g|=  7.52749D-04

At iterate   20    f=  1.37230D+00    |proj g|=  1.34413D-03

At iterate   25    f=  1.37230D+00    |proj g|=  4.48013D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     28     34      1     0     0   7.643D-06   1.372D+00
  F =   1.3723005599156413     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

 This problem is unconstrained.


...


At iterate    5    f=  1.31178D+00    |proj g|=  4.50936D-03

At iterate   10    f=  1.31090D+00    |proj g|=  5.37287D-03

At iterate   15    f=  1.31071D+00    |proj g|=  3.79664D-03

At iterate   20    f=  1.31069D+00    |proj g|=  1.02098D-03

At iterate   25    f=  1.31069D+00    |proj g|=  8.73346D-04

At iterate   30    f=  1.31069D+00    |proj g|=  6.32581D-05

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     31     39      1     0     0   3.697D-05   1.311D+00
  F =   1.3106858355388937     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machin

 This problem is unconstrained.



At iterate    5    f=  4.65030D-01    |proj g|=  1.03983D-01

At iterate   10    f=  4.41945D-01    |proj g|=  2.67046D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     13     21      1     0     0   2.205D-05   4.419D-01
  F =  0.44194391170793923     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.17658D+00    |proj g|=  2.82964D-01


 This problem is unconstrained.



At iterate    5    f=  1.05714D+00    |proj g|=  5.36202D-03

At iterate   10    f=  1.05567D+00    |proj g|=  9.66966D-03

At iterate   15    f=  1.05540D+00    |proj g|=  2.49818D-03

At iterate   20    f=  1.05537D+00    |proj g|=  1.11959D-03

At iterate   25    f=  1.05537D+00    |proj g|=  4.18887D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3     26     40      1     0     0   4.207D-04   1.055D+00
  F =   1.0553662479560921     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [21]:
#Prophet

dates = ["2007-12-" + str(i) for i in range(1, 31)]
predictions = []
for row in tqdm(train_dataset[train_dataset.columns[-30:]].values[:n_rows]):
    df = pd.DataFrame(np.transpose([dates, row]))
    df.columns = ["ds", "y"]
    model = Prophet(daily_seasonality=True)
    model.fit(df)
    future = model.make_future_dataframe(periods=30)
    forecast = model.predict(future)["yhat"].loc[30:].values
    predictions.append(forecast)
predictions = np.array(predictions).reshape((-1, 30))
error_prophet = np.linalg.norm(predictions[:n_rows] - val_dataset.values[:n_rows])/len(predictions[0])


  0%|          | 0/25 [00:00<?, ?it/s]

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.



Initial log joint probability = -28.9301
Iteration  1. Log joint probability =    9.18298. Improved by 38.113.
Iteration  2. Log joint probability =    25.4207. Improved by 16.2377.
Iteration  3. Log joint probability =    26.0977. Improved by 0.676986.
Iteration  4. Log joint probability =    26.3816. Improved by 0.283952.
Iteration  5. Log joint probability =    26.3958. Improved by 0.0141368.
Iteration  6. Log joint probability =    26.4112. Improved by 0.0154235.
Iteration  7. Log joint probability =    26.4137. Improved by 0.00253877.
Iteration  8. Log joint probability =    26.4214. Improved by 0.00766101.
Iteration  9. Log joint probability =    26.4299. Improved by 0.00854073.
Iteration 10. Log joint probability =    26.4331. Improved by 0.0031661.
Iteration 11. Log joint probability =    26.4368. Improved by 0.0037265.
Iteration 12. Log joint probability =    26.4384. Improved by 0.00158862.
Iteration 13. Log joint probability =    26.4395. Improved by 0.00107142.
Iteration 1

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.



Initial log joint probability = -30.3946
Iteration  1. Log joint probability =     3.7117. Improved by 34.1063.
Iteration  2. Log joint probability =    10.0115. Improved by 6.29977.
Iteration  3. Log joint probability =    15.8254. Improved by 5.81397.
Iteration  4. Log joint probability =    16.0828. Improved by 0.257403.
Iteration  5. Log joint probability =    16.0901. Improved by 0.00730186.
Iteration  6. Log joint probability =    16.1216. Improved by 0.0314168.
Iteration  7. Log joint probability =    16.1288. Improved by 0.00724291.
Iteration  8. Log joint probability =    16.2066. Improved by 0.0778419.
Iteration  9. Log joint probability =    16.4858. Improved by 0.279201.
Iteration 10. Log joint probability =    16.5475. Improved by 0.0616134.
Iteration 11. Log joint probability =    16.5752. Improved by 0.0276983.
Iteration 12. Log joint probability =       16.6. Improved by 0.0248926.
Iteration 13. Log joint probability =    16.6872. Improved by 0.0871808.
Iteration 14. L

...


Initial log joint probability = -28.5466
Iteration  1. Log joint probability =    17.0992. Improved by 45.6457.
Iteration  2. Log joint probability =    17.6784. Improved by 0.579234.
Iteration  3. Log joint probability =    25.0039. Improved by 7.32552.
Iteration  4. Log joint probability =    25.5125. Improved by 0.508531.
Iteration  5. Log joint probability =    25.6373. Improved by 0.124808.
Iteration  6. Log joint probability =     25.681. Improved by 0.0437768.
Iteration  7. Log joint probability =    25.7475. Improved by 0.0664306.
Iteration  8. Log joint probability =    25.7582. Improved by 0.0107595.
Iteration  9. Log joint probability =    25.7927. Improved by 0.0344381.
Iteration 10. Log joint probability =    25.8013. Improved by 0.00860814.
Iteration 11. Log joint probability =    25.8233. Improved by 0.0219971.
Iteration 12. Log joint probability =    25.8282. Improved by 0.00493799.
Iteration 13. Log joint probability =    25.8805. Improved by 0.0522575.
Iteration 14. 

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.



Initial log joint probability = -31.3617
Iteration  1. Log joint probability =    14.6387. Improved by 46.0004.
Iteration  2. Log joint probability =    31.7274. Improved by 17.0887.
Iteration  3. Log joint probability =    37.8263. Improved by 6.09892.
Iteration  4. Log joint probability =      38.01. Improved by 0.183657.
Iteration  5. Log joint probability =    38.0219. Improved by 0.0119737.
Iteration  6. Log joint probability =    38.0401. Improved by 0.0181185.
Iteration  7. Log joint probability =     38.087. Improved by 0.046952.
Iteration  8. Log joint probability =    38.1195. Improved by 0.0324449.
Iteration  9. Log joint probability =    38.1623. Improved by 0.0428312.
Iteration 10. Log joint probability =    38.1807. Improved by 0.0184416.
Iteration 11. Log joint probability =    38.1893. Improved by 0.00858192.
Iteration 12. Log joint probability =    38.1925. Improved by 0.00316915.
Iteration 13. Log joint probability =    38.1965. Improved by 0.00400616.
Iteration 14. 

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 23.



Initial log joint probability = -30.3617
Iteration  1. Log joint probability =    10.2333. Improved by 40.595.
Iteration  2. Log joint probability =    17.5574. Improved by 7.32411.
Iteration  3. Log joint probability =    17.8316. Improved by 0.274254.
Iteration  4. Log joint probability =    17.8595. Improved by 0.0278989.
Iteration  5. Log joint probability =    18.0078. Improved by 0.148284.
Iteration  6. Log joint probability =    18.0126. Improved by 0.00480151.
Iteration  7. Log joint probability =    18.0195. Improved by 0.00689045.
Iteration  8. Log joint probability =    18.0229. Improved by 0.00343353.
Iteration  9. Log joint probability =    18.0256. Improved by 0.00264141.
Iteration 10. Log joint probability =     18.028. Improved by 0.0024202.
Iteration 11. Log joint probability =    18.0294. Improved by 0.00143387.
Iteration 12. Log joint probability =      18.03. Improved by 0.000606135.
Iteration 13. Log joint probability =    18.0301. Improved by 5.37927e-05.
Iterati

In [22]:
error = [error_naive, error_avg, error_holt, error_exponential, error_arima, error_prophet]
names = ["Naive approach", "Moving average", "Holt linear", "Exponential smoothing", "ARIMA", "Prophet"]
df = pd.DataFrame(np.transpose([error, names]))
df.columns = ["RMSE Loss", "Model"]
df = df.sort_values(by=['RMSE Loss'])
px.bar(df, y="RMSE Loss", x="Model", color="Model", title="RMSE Loss vs. Model")

In [11]:
#Generate predictions file

days = range(1, 1913 + 1)
time_series_columns = [f'd_{i}' for i in days]
time_series_data = sales_train_val[time_series_columns]
forecast = pd.DataFrame(time_series_data.iloc[:, -28:].mean(axis=1))
forecast = pd.concat([forecast] * 28, axis=1)
forecast.columns = [f'F{i}' for i in range(1, forecast.shape[1] + 1)]
validation_ids = sales_train_val['id'].values
evaluation_ids = [i.replace('validation', 'evaluation') for i in validation_ids]
ids = np.concatenate([validation_ids, evaluation_ids])
predictions = pd.DataFrame(ids, columns=['id'])
forecast = pd.concat([forecast] * 2).reset_index(drop=True)
predictions = pd.concat([predictions, forecast], axis=1)
predictions.to_csv('submission.csv', index=False)